手写数字图像的分类，直接使用学习到的参数，实现神经网络的推理处理，也叫做“前向传播”。

使用神经网络解决问题时，需要首先使用训练数据进行权重参数的学习，进行推理时，使用刚才学习的参数，对输入图像进行分类。

MNIST数据集：
    1、MNIST数据集是由美国国家标准与技术研究所（NIST）收集的手写数字图像数据集。
    2、MNIST数据集是由60000个训练图像和10000个测试图像组成的。
    3、MNIST数据集的图像是28x28像素的灰度图像，每个像素的取值范围是0.0到1.0，0.0表示白色，1.0表示黑色。
    4、MNIST数据集的标签是0到9的10个数字，对应10个类别。


load_mnist 函数以 （训练图像，训练标签），（测试图像，测试标签）的形式返回读入的MNIST数据。

flatten=  ： 设置是否展开输入图像（变成一维数组），False ：不展开，输入图像为1x28x28的三维数组；True:则输入图像会保存为由784个元素构成的一维数组
normalize= ：设置是否正规化，False：不正规化，取值范围为0-255；True：正规化，取值范围为0.0-1.0
one_hot_label=True ： 进行独热编码（one-hot encoding），one_hot表示是仅正解的标签为1，其余都是0 的数组如[0,0,1,0,0,0,0,0,0,0],False,只是像7，2这样简单保存正确解标签。



In [5]:
import sys,os
import numpy as np
sys.path.append(os.pardir)   #为了导入父目录的文件而进行的设定
from digit_recognition.mnist import load_mnist
#第二次及以后的调用只需读入保存在本地的文件（pickle文件）即可
(x_train,t_train),(x_test,t_test)=load_mnist(flatten=True,normalize=False)
#输出训练数据的形状
print(x_train.shape)
#输出训练标签的形状
print(t_train.shape)
#输出测试数据的形状
print(x_test.shape)
#输出测试标签的形状
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


显示一张MNIST图像

In [6]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
from digit_recognition.mnist import load_mnist
from PIL import Image


def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)  # 5

print(img.shape)  # (784,)
img = img.reshape(28, 28)  # 把图像的形状变为原来的尺寸
print(img.shape)  # (28, 28)

img_show(img)



5
(784,)
(28, 28)


神经网络的推理处理：

对于一张图像来说 神经网络的输入层有784个神经元，输出层有10个神经元。输入图像的每个像素对应神经网络的输入层的一个神经元。
    输入层的784这个数字来源于图像的大小。图像是28*28像素的，所以输入层的神经元有784个。
    输出层的10这个数字来源于10个类别分类（数字0-9，共10类别）

设置这个神经网络有2个隐藏层，第1个隐藏层有50个神经元，第2个隐藏层有100个神经元。（50和100可以设置成任何值）

In [9]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
import pickle
from digit_recognition.mnist import load_mnist
from PIL import Image


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

#normalize=True将图像的各个像素值除以255，使得数据的值在0.0~1.0之间
#flatten=True将图像展开为一维数组
#one_hot_label=False将标签作为数字（0-9）返回，而不是one-hot数组
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    with open("./digit_recognition/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

"""
以numpy数组形式输出各个标签对应的概率,比如输出[0.1,0.2,0.3,...0.4]表示数字0的概率为0.1,数字1的概率为0.2,数字2的概率为0.3,数字9的概率为0.4,
然后取出这个概率列表中的最大值的索引（第几个元素的概率最高）作为预测结果。np.argmax(y)取出数组中最大值的索引 。
"""
x, t = get_data()
network = init_network()
accuracy_cnt = 0
print(x.shape,len(x))
print(t.shape)
for i in range(len(x)):
    y = predict(network, x[i])
    p= np.argmax(y) # 获取概率最高的元素的索引
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))   #表示有93.52%的图片被正确分类了

(10000, 784) 10000
(10000,)
Accuracy:0.9352


批处理

输出以上神经网络的各层权重的形状

In [10]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']
print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)


(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [11]:
list(range(0, 10, 3))
x=np.array([[0.1, 0.2, 0.3],[0.1, 0.4, 0.3],[0.1, 0.2, 0.6],[0.1, 0.8, 0.3]])
y=np.argmax(x, axis=1)
print(y)


[2 1 2 1]


In [12]:
y=np.array([0, 3, 2, 3])
t=np.array([0, 1, 2, 3])
print(y==t)
print(np.sum(y==t))


[ True False  True  True]
3


In [14]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
import pickle
from digit_recognition.mnist import load_mnist

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x - c)
    sum_exp_x = np.sum(exp_x)
    y = exp_x / sum_exp_x
    return y


def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    with open("./digit_recognition/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, w3) + b3
    y = softmax(a3)

    return y


x, t = get_data()
network = init_network()

batch_size = 100 # 批数量
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))


Accuracy:0.9352
